In [136]:
%matplotlib inline
from pprint import pprint

import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [137]:
RAND_SEED = 42
np.random.seed(RAND_SEED)

In [138]:
df_house = pd.read_csv("DATA1_housing.csv")
df_passangers = pd.read_csv("DATA2_passengers.csv")

In [139]:
df_house.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [140]:
df_passangers.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# DATA frame passangers 

In [141]:
df_passangers.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Age,8514.0,28.827930,14.489021,0.0,19.0,27.0,38.0,79.0
RoomService,8512.0,224.687617,666.717663,0.0,0.0,0.0,47.0,14327.0
FoodCourt,8510.0,458.077203,1611.489240,0.0,0.0,0.0,76.0,29813.0
ShoppingMall,8485.0,173.729169,604.696458,0.0,0.0,0.0,27.0,23492.0
Spa,8510.0,311.138778,1136.705535,0.0,0.0,0.0,59.0,22408.0
VRDeck,8505.0,304.854791,1145.717189,0.0,0.0,0.0,46.0,24133.0


In [142]:
df_passangers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [143]:
df_sliced=df_passangers[df_passangers["Transported"].notna()]
df_sliced.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [144]:

df_sliced.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [145]:
df_sliced.groupby(by='HomePlanet')["Transported"].value_counts(normalize=True)

HomePlanet  Transported
Earth       False          0.576054
            True           0.423946
Europa      True           0.658846
            False          0.341154
Mars        True           0.523024
            False          0.476976
Name: Transported, dtype: float64

In [147]:
df_sliced.groupby(by='CryoSleep')["Transported"].value_counts(normalize=True)

CryoSleep  Transported
False      False          0.671079
           True           0.328921
True       True           0.817583
           False          0.182417
Name: Transported, dtype: float64

In [148]:
df_sliced.groupby(by='Cabin')["Transported"].value_counts(normalize=True)

Cabin   Transported
A/0/P   False          0.5
        True           0.5
A/0/S   False          1.0
A/1/S   True           1.0
A/10/P  False          1.0
                      ... 
T/0/P   False          1.0
T/1/P   False          1.0
T/2/P   False          1.0
T/2/S   False          1.0
T/3/P   True           1.0
Name: Transported, Length: 7169, dtype: float64

In [149]:
df_sliced.groupby(by='Destination')["Transported"].value_counts(normalize=True)

Destination    Transported
55 Cancri e    True           0.610000
               False          0.390000
PSO J318.5-22  True           0.503769
               False          0.496231
TRAPPIST-1e    False          0.528825
               True           0.471175
Name: Transported, dtype: float64

# Como fazer um od.crosstab de value counts para ter a frequencia relativa

In [159]:
df_sliced['Faixa_idade']=pd.cut(df_sliced['Age'],np.arange(0,85,5))
oi=df_sliced['Faixa_idade'].value_counts(normalize=True)
print(oi)
pd.crosstab(df_sliced['Transported'],df_sliced['Faixa_idade'])

(20, 25]    0.175264
(25, 30]    0.144794
(15, 20]    0.143354
(30, 35]    0.107726
(35, 40]    0.093810
(40, 45]    0.069578
(10, 15]    0.062620
(45, 50]    0.049664
(0, 5]      0.038508
(50, 55]    0.036828
(5, 10]     0.026272
(55, 60]    0.025192
(60, 65]    0.016315
(65, 70]    0.005638
(70, 75]    0.003239
(75, 80]    0.001200
Name: Faixa_idade, dtype: float64


Faixa_idade,"(0, 5]","(5, 10]","(10, 15]","(15, 20]","(20, 25]","(25, 30]","(30, 35]","(35, 40]","(40, 45]","(45, 50]","(50, 55]","(55, 60]","(60, 65]","(65, 70]","(70, 75]","(75, 80]"
Transported,,,,,,,,,,,,,,,,
False,87,89,226,602,808,608,480,413,290,209,164,100,72,21,16,7
True,234,130,296,593,653,599,418,369,290,205,143,110,64,26,11,3


In [160]:
df_sliced.groupby(by='VIP')["Transported"].value_counts(normalize=True)

VIP    Transported
False  True           0.506332
       False          0.493668
True   False          0.618090
       True           0.381910
Name: Transported, dtype: float64

In [165]:
#nao sei fazer uma analise com esses valores
df_sliced["ShoppingMall"].describe()
df_sliced["FoodCourt"].describe()
df_sliced[""]

count     8510.000000
mean       458.077203
std       1611.489240
min          0.000000
25%          0.000000
50%          0.000000
75%         76.000000
max      29813.000000
Name: FoodCourt, dtype: float64

In [164]:
df_sliced.corr()["Transported"].sort_values(ascending=False).drop("Transported")

FoodCourt       0.046566
ShoppingMall    0.010141
Age            -0.075026
VRDeck         -0.207075
Spa            -0.221131
RoomService    -0.244611
Name: Transported, dtype: float64

# Posso pegar correlação de um valor boleano?
.

## colunas com boa correlação


In [ ]:
BINS= int(np.sqrt(len(df_sliced.index)))
df_sliced.hist(bins=BINS,figsize=(15,10));

In [ ]:
#df_sliced["Spa"].apply(np.log10).plot.hist(bins=BINS)

Contando a quantidade de passageiros que a gente vai perder caso aplique esses cortes: 

In [ ]:
df_sliced.count()

In [ ]:

df_sliced[df_sliced["RoomService"]>4000].loc[:,"RoomService"].count()

In [ ]:
df_sliced[df_sliced["FoodCourt"]>8000].loc[:,"FoodCourt"].count()


In [ ]:
df_sliced[df_sliced["Spa"]>5000].loc[:,"Spa"].count()

In [ ]:
df_sliced[df_sliced["ShoppingMall"]>2500].loc[:,"ShoppingMall"].count()

In [ ]:
df_sliced[df_sliced["VRDeck"]>5500].loc[:,"VRDeck"].count()

In [ ]:
df_dropped=df_sliced
idx = (df_dropped["RoomService"]<=4000)\
    & (df_dropped["FoodCourt"] <=8000) \
    & (df_dropped["Spa"] <=5000) \
    & (df_dropped["ShoppingMall"]<= 2500 ) \
    & (df_dropped["VRDeck"] <=5500) 
df_filtered = df_dropped[idx].copy()
df_filtered.info()

In [ ]:
#(df_filtered["ShoppingMall"]!=0).apply(np.log10).plot.hist(bins=BINS)


# Data frame housing

In [ ]:
df_house.describe().transpose()

In [ ]:
df_house.info()

In [ ]:
df=df_house[df_house["median_house_value"].notna()]
df.head()

In [ ]:
df.corr()["median_house_value"].sort_values(ascending=False).drop("median_house_value")

In [ ]:
df.info()

In [ ]:
BINS2=int(np.sqrt(len(df.index)))

In [ ]:
df.hist(bins=BINS,figsize=(15,10));

In [ ]:
df["total_rooms"].apply(np.log10).plot.hist(bins=BINS2)

In [ ]:
df[df["total_rooms"]<500].loc[:,"total_rooms"].count()

In [ ]:
df[["total_rooms"]>=500 & ].loc[:,"total_rooms"].apply(np.log10).plot.hist(bins=BINS2)